In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bengaliai-cv19/test_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test_image_data_3.parquet
/kaggle/input/bengaliai-cv19/test_image_data_0.parquet
/kaggle/input/bengaliai-cv19/train.csv
/kaggle/input/bengaliai-cv19/test_image_data_1.parquet
/kaggle/input/bengaliai-cv19/class_map.csv
/kaggle/input/bengaliai-cv19/train_image_data_3.parquet
/kaggle/input/bengaliai-cv19/train_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test.csv
/kaggle/input/bengaliai-cv19/sample_submission.csv
/kaggle/input/bengaliai-cv19/train_image_data_1.parquet
/kaggle/input/bengaliai-cv19/train_image_data_0.parquet


## 概要  
このKernelでは実際にCNNのモデルを作っていく。  
CNNの構造は以下のKernelを参考にした。  
https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn  
最初のモデルなので、データの内容についても詳しく説明していく。

In [2]:
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2

from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,BatchNormalization, Input,Activation
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns

Using TensorFlow backend.


## データのinput  


In [3]:
train_df_ = pd.read_csv('/kaggle/input/bengaliai-cv19/train.csv')
test_df_ = pd.read_csv('/kaggle/input/bengaliai-cv19/test.csv')
class_map_df = pd.read_csv('/kaggle/input/bengaliai-cv19/class_map.csv')
sample_sub_df = pd.read_csv('/kaggle/input/bengaliai-cv19/sample_submission.csv')

train_df_ = train_df_.drop(['grapheme'], axis=1, inplace=False) #grapheme(実際の文字)のデータはいらない

## 画像サイズの設定  
学習時間と精度に影響する。N_CHANNELSは3次元方向には1なので、1を設定。convolution層で32になったりする。

In [4]:
IMG_SIZE=64
N_CHANNELS=1

## resize  
元画像からROIを中央に設定し、引数sizeのサイズに圧縮する。  
tqdmはプログレスバーを表示するライブラリ。dfの行数を入れてやる  
cv2はopenCVのことで、画像処理ライブラリ。  
_,thresh = cv2.threshold(画像,閾値,閾値の画素の設定値,手法)  
cv2.thresholdは2つの値を返す。1つめは手法により探索した閾値で、2つめは閾値を適応した画像を返す。  
今回は閾値を適応した二値化画像がほしいので、第一返り値は_であしらっている。  
手法についてだが、大津の二値化と反転二値化(parquetファイルを見れば分かるが、背景が255などの高い画素値、文字が0などの低い画素値で入っている為、反転する必要がある。)で閾値を求め、その合計を返している。下記URLより２手法による閾値の設定の例を見れば分かるが、よりくっきりとした画像を得ることができる。  
http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html  

contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]  
findContoursは輪郭の検出を行う関数である。第一引数に先程作成した二値化画像を用いている。第二引数(cv2.RETR_LIST)はcontour retrieval mode，第３引数は輪郭検出方法(cv2.CHAIN_APPROX_SIMPLE)を指定するフラグである。CHAIN_APPROX_SIMPLEは簡易的に輪郭の特徴となる点を保持する手法(輪郭の近似)であり、これにより線ではなく点で情報を保持することでデータ量を削減できる。  
[-2:]についてだが、難しく考える必要はない。  
本当はfindContoursは3つの値を返す関数だが、アンダースコアを2つ用意するのが面倒なため、1つめの返り値を端折っている。ちなみに、3つの返り値は順番に輪郭画像(list)，輪郭(list)，輪郭の階層情報である。  
http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html?highlight=cv2%20findcontours  

x,y,w,h = cv2.boundingRect(cnt)  
boundingRectはcontoursのデータを元に、その輪郭の四方の座標を取得(外接矩形)する。
cntには先程求めたcontoursの情報が入っている。  
戻り値は、画像の左上を(x,y)として、そこから右にw,下にhだけ移動した長方形が輪郭の外接矩形となる。  
contoursには複数の輪郭情報が入る場合があるため、それぞれの最大値をROIとして設定する端の情報として保持する。  
http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html?highlight=cv2%20boundingrect  

resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)  
cv2.resizeは画像を補完しながら引き伸ばす、あるいは収縮する関数である。これをスケーリングという。  
今回は画像を(64x64)にする。ROIを加味しても元画像より画像サイズが小さくなると予想される為、interpolation(スケーリング手法)には収縮に適したINTER_AREAを採用している。  



In [5]:
def resize(df, size=IMG_SIZE, need_progress_bar=True):
    resized = {}
    resize_size=IMG_SIZE
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])): #tqdm...プログレスバーを表示するライブラリ。dfの行数を入れてやる
            image=df.loc[df.index[i]].values.reshape(137,236) #dfのi行目を1次元→137x236の形に変える
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU) #しきい値の設定
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:] #輪郭の検出

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours: #求めた輪郭情報を用いて、画像上の文字が書いてある部分の端の座標を求める。
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    else: #プログレスバーを表示しないこと以外は同じ
        for i in range(df.shape[0]):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    resized = pd.DataFrame(resized).T #現在、valueに画像情報が入っているため、そのままdf型にすると行方向に画像データが入る。そのため転置して行→列にしている。
    return resized

## OneHotEncoding  
ダミー関数を取得する関数を定義する

In [6]:
def get_dummies(df):
    cols = []
    for col in df:
        cols.append(pd.get_dummies(df[col].astype(str)))
    return pd.concat(cols, axis=1)

## モデルの定義  
4層の畳み込み層→正規化→プーリング層→畳み込み層→ドロップアウト　これを4回繰り返すのが基本的な構造になっている。
出力層ではFlatten関数で1次元化し、全結合層(Dense)によって1024個のデータに圧縮し、さらにドロップアウトを通して全結合層で512個のデータに圧縮する。これをdenseという変数に格納することで、書記素、母音、子音それぞれのサイズに合わせた全結合層を用意し、それらをアウトプット(分類問題なので活性化関数はsoftmax)する。  

In [7]:
def vgg19():
    inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1)) #keras.layers.Input()

    # Block 1
    conv1_1 = Conv2D(64, (3, 3),name='conv1_1', activation='relu', padding='same')(inputs)
    conv1_2 = Conv2D(64, (3, 3),name='conv1_2', activation='relu', padding='same')(conv1_1)
    bn1 = BatchNormalization(axis=3)(conv1_2)
    pool1 = MaxPooling2D(pool_size=(2, 2))(bn1)
    drop1 = Dropout(0.5)(pool1)

    # Block 2
    conv2_1 = Conv2D(128, (3, 3),name='conv2_1', activation='relu', padding='same')(drop1)
    conv2_2 = Conv2D(128, (3, 3),name='conv2_2', activation='relu', padding='same')(conv2_1)
    bn2 = BatchNormalization(axis=3)(conv2_2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(bn2)
    drop2 = Dropout(0.5)(pool2)

    # Block 3
    conv3_1 = Conv2D(256, (3, 3),name='conv3_1', activation='relu', padding='same')(drop2)
    conv3_2 = Conv2D(256, (3, 3),name='conv3_2', activation='relu', padding='same')(conv3_1)
    conv3_3 = Conv2D(256, (3, 3),name='conv3_3', activation='relu', padding='same')(conv3_2)
    conv3_4 = Conv2D(256, (3, 3),name='conv3_4', activation='relu', padding='same')(conv3_3)
    bn3 = BatchNormalization(axis=3)(conv3_4)
    pool3 = MaxPooling2D(pool_size=(2, 2))(bn3)
    drop3 = Dropout(0.5)(pool3)

    # Block 4
    conv4_1 = Conv2D(512, (3, 3),name='conv4_1', activation='relu', padding='same')(drop3)
    conv4_2 = Conv2D(512, (3, 3),name='conv4_2', activation='relu', padding='same')(conv4_1)
    conv4_3 = Conv2D(512, (3, 3),name='conv4_3', activation='relu', padding='same')(conv4_2)
    conv4_4 = Conv2D(512, (3, 3),name='conv4_4', activation='relu', padding='same')(conv4_3)
    bn4 = BatchNormalization(axis=3)(conv4_4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(bn4)
    drop4 = Dropout(0.5)(pool4)

    # Block 5
    conv5_1 = Conv2D(512, (3, 3),name='conv5_1', activation='relu', padding='same')(drop4)
    conv5_2 = Conv2D(512, (3, 3),name='conv5_2', activation='relu', padding='same')(conv5_1)
    conv5_3 = Conv2D(512, (3, 3),name='conv5_3', activation='relu', padding='same')(conv5_2)
    conv5_4 = Conv2D(512, (3, 3),name='conv5_4', activation='relu', padding='same')(conv5_3)
    bn5 = BatchNormalization(axis=3)(conv5_4)
    pool5 = MaxPooling2D(pool_size=(2, 2))(bn5)
    drop5 = Dropout(0.5)(pool5)
    
    x = Flatten()(drop5)
    x = Dense(1024)(x)
    #x = Activation('relu')(x)
    #ここから加工
    model = Dropout(rate=0.3)(x)
    dense = Dense(512, activation = "relu")(model) #全結合層2

    head_root = Dense(168, activation = 'softmax')(dense) #全結合層3(from全結合層2)-書記素
    head_vowel = Dense(11, activation = 'softmax')(dense) #全結合層4(from全結合層2)-母音分音記号
    head_consonant = Dense(7, activation = 'softmax')(dense) #全結合層5(from全結合層2)-子音分音記号

    
    model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])
    return model

model = vgg19()

In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 64, 64, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 64, 64, 64)   36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 64)   256         conv1_2[0][0]                    
____________________________________________________________________________________________

## コンパイル  
モデルと最適化手法を結びつける。  
最適化手法はAdam(その他、RMSprop,SGDなど)、損失関数はcategorical_crossentropy、評価関数は精度(accuracy)。  

※categorical_crossentropyの説明：  
カテゴリIndexで与えられるデータセットの変数との交差エントロピーを最小化するニューラルネットワークの出力層です。

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 学習率の設定  
3世代の学習(patience=3)で精度が向上しない場合、50%の学習率に減少する(factor=0.5)。但し、最低の学習率を0.001%にする(min_lr=0.00001)。  


monitor: 監視する値．  
•factor: 学習率を減らす割合．new_lr = lr * factor  
•patience: 何エポック改善が見られなかったら学習率の削減を行うか．  
•verbose: 整数．0: 何も表示しない．1: 学習率削減時メッセージを表示．  
•mode: auto，min，maxのいずれか． minの場合，監視する値の減少が停止した際に，学習率を更新します． maxの場合，監視する値の増加が停止した時に，学習率を更新します． autoの場合，監視する値の名前から自動で判断します．  
•epsilon: 改善があったと判断する閾値．有意な変化だけに注目するために用います．  
•cooldown: 学習率を減らした後，通常の学習を再開するまで待機するエポック数．  
•min_lr: 学習率の下限．  
https://keras.io/ja/callbacks/

In [10]:
# Set a learning rate annealer. Learning rate will be half after 3 epochs if accuracy is not increased
learning_rate_reduction_root = ReduceLROnPlateau(monitor='dense_3_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='dense_4_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='dense_5_accuracy', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)

## バッチサイズと世代の設定

In [11]:
batch_size = 512
epochs = 30

## 学習の実行  
train_image_dataが4つ(0~4)に分かれているので、ループで回す。

In [12]:
histories = []
for i in range(4):
    train_df = pd.merge(pd.read_parquet(f'/kaggle/input/bengaliai-cv19/train_image_data_{i}.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)
    
    X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1)
    X_train = resize(X_train)/255
    
    # CNN takes images in shape `(batch_size, h, w, channels)`, so reshape the images
    X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
    Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
    Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values

    print(f'Training images: {X_train.shape}')
    print(f'Training labels root: {Y_train_root.shape}')
    print(f'Training labels vowel: {Y_train_vowel.shape}')
    print(f'Training labels consonants: {Y_train_consonant.shape}')

    # Divide the data into training and validation set
    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=666)
    del train_df
    del X_train
    del Y_train_root, Y_train_vowel, Y_train_consonant

    # Fit the model
    history = model.fit(x_train, {'dense_3': y_train_root, 'dense_4': y_train_vowel, 'dense_5': y_train_consonant}, batch_size=batch_size,
                              epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                              #steps_per_epoch=x_train.shape[0] // batch_size, 
                              callbacks=[learning_rate_reduction_root, learning_rate_reduction_vowel, learning_rate_reduction_consonant])

    histories.append(history)
    
    # Delete to reduce memory usage
    del x_train
    del x_test
    del y_train_root
    del y_test_root
    del y_train_vowel
    del y_test_vowel
    del y_train_consonant
    del y_test_consonant
    gc.collect()


Training images: (50210, 64, 64, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Train on 46193 samples, validate on 4017 samples
Epoch 1/30
46193/46193 [==============================] - 57s 1ms/step - loss: 9.0529 - dense_3_loss: 5.0538 - dense_4_loss: 2.4257 - dense_5_loss: 1.5653 - dense_3_accuracy: 0.0247 - dense_4_accuracy: 0.1962 - dense_5_accuracy: 0.5945 - val_loss: 8.3264 - val_dense_3_loss: 4.8538 - val_dense_4_loss: 2.1484 - val_dense_5_loss: 1.3249 - val_dense_3_accuracy: 0.0319 - val_dense_4_accuracy: 0.2173 - val_dense_5_accuracy: 0.6107
Epoch 2/30
46193/46193 [==============================] - 47s 1ms/step - loss: 7.7358 - dense_3_loss: 4.7027 - dense_4_loss: 1.8987 - dense_5_loss: 1.1323 - dense_3_accuracy: 0.0322 - dense_4_accuracy: 0.2975 - dense_5_accuracy: 0.6236 - val_loss: 8.3151 - val_dense_3_loss: 4.7335 - val_dense_4_loss: 2.3056 - val_dense_5_loss: 1.2759 - val_dense_3_accuracy: 0.0224 - val_den


Training images: (50210, 64, 64, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Train on 46193 samples, validate on 4017 samples
Epoch 1/30
46193/46193 [==============================] - 47s 1ms/step - loss: 0.8845 - dense_3_loss: 0.5875 - dense_4_loss: 0.1593 - dense_5_loss: 0.1358 - dense_3_accuracy: 0.8448 - dense_4_accuracy: 0.9592 - dense_5_accuracy: 0.9618 - val_loss: 0.6447 - val_dense_3_loss: 0.4249 - val_dense_4_loss: 0.1190 - val_dense_5_loss: 0.1015 - val_dense_3_accuracy: 0.8885 - val_dense_4_accuracy: 0.9724 - val_dense_5_accuracy: 0.9746
Epoch 2/30
46193/46193 [==============================] - 47s 1ms/step - loss: 0.6352 - dense_3_loss: 0.4215 - dense_4_loss: 0.1098 - dense_5_loss: 0.1031 - dense_3_accuracy: 0.8843 - dense_4_accuracy: 0.9712 - dense_5_accuracy: 0.9716 - val_loss: 0.6468 - val_dense_3_loss: 0.3938 - val_dense_4_loss: 0.1537 - val_dense_5_loss: 0.0978 - val_dense_3_accuracy: 0.8964 - val_den


Training images: (50210, 64, 64, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Train on 46193 samples, validate on 4017 samples
Epoch 1/30
46193/46193 [==============================] - 47s 1ms/step - loss: 0.6574 - dense_3_loss: 0.4328 - dense_4_loss: 0.1134 - dense_5_loss: 0.1126 - dense_3_accuracy: 0.9136 - dense_4_accuracy: 0.9786 - dense_5_accuracy: 0.9771 - val_loss: 0.4827 - val_dense_3_loss: 0.2978 - val_dense_4_loss: 0.0819 - val_dense_5_loss: 0.1034 - val_dense_3_accuracy: 0.9373 - val_dense_4_accuracy: 0.9846 - val_dense_5_accuracy: 0.9764
Epoch 2/30
46193/46193 [==============================] - 47s 1ms/step - loss: 0.4677 - dense_3_loss: 0.3107 - dense_4_loss: 0.0789 - dense_5_loss: 0.0785 - dense_3_accuracy: 0.9240 - dense_4_accuracy: 0.9819 - dense_5_accuracy: 0.9800 - val_loss: 0.4175 - val_dense_3_loss: 0.2591 - val_dense_4_loss: 0.0754 - val_dense_5_loss: 0.0836 - val_dense_3_accuracy: 0.9425 - val_den


Training images: (50210, 64, 64, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Train on 46193 samples, validate on 4017 samples
Epoch 1/30
46193/46193 [==============================] - 47s 1ms/step - loss: 0.5050 - dense_3_loss: 0.3359 - dense_4_loss: 0.0850 - dense_5_loss: 0.0831 - dense_3_accuracy: 0.9252 - dense_4_accuracy: 0.9813 - dense_5_accuracy: 0.9804 - val_loss: 0.4002 - val_dense_3_loss: 0.2484 - val_dense_4_loss: 0.0818 - val_dense_5_loss: 0.0677 - val_dense_3_accuracy: 0.9450 - val_dense_4_accuracy: 0.9826 - val_dense_5_accuracy: 0.9841
Epoch 2/30
46193/46193 [==============================] - 47s 1ms/step - loss: 0.3952 - dense_3_loss: 0.2632 - dense_4_loss: 0.0679 - dense_5_loss: 0.0666 - dense_3_accuracy: 0.9325 - dense_4_accuracy: 0.9835 - dense_5_accuracy: 0.9825 - val_loss: 0.3749 - val_dense_3_loss: 0.2325 - val_dense_4_loss: 0.0785 - val_dense_5_loss: 0.0621 - val_dense_3_accuracy: 0.9427 - val_den

## Validation (prediction)  
学習が終わったmodelについて、model.predictでValidationを行う。これの返り値にはそれぞれのcomponentsの確率がリストとして入っている。  
これをnp.argmax()によって、softmax関数によって確率化された値の中の最も大きい値のindexをpreds_dictに入れる。
各テストデータの各要素についてループを回し、targetに値を入れ、DataFrame型に変換しoutput。

In [13]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

In [14]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet('/kaggle/input/bengaliai-cv19/test_image_data_{}.parquet'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)

    X_test = resize(df_test_img, need_progress_bar=False)/255 #test画像の前処理を行う。
    X_test = X_test.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
    preds = model.predict(X_test) #Valdiation

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1) #最も確率が高いものを採用(回帰(確率)→分類)

    for k,id in enumerate(df_test_img.index.values): #それぞれのidについて
        for i,comp in enumerate(components): #3要素を回して
            id_sample=id+'_'+comp #列名を作成し、
            row_id.append(id_sample)
            target.append(preds_dict[comp][k]) #targetに3つの要素の予測値をcomponentsの順番で入れていく
    del df_test_img
    del X_test
    gc.collect()

df_sample = pd.DataFrame( #DataFrame型にsubmissionデータを格納し、
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False) #csvに変換。
df_sample.head()

,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,3
2,Test_0_vowel_diacritic,0
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,93


<a href="Your file path"> Download File </a>